## Interface 

Single particle close to an interface

    interfaceME.py

In [1]:
from sympy import *
import numpy as np
import re
from joblib import Parallel, delayed

### Definitions and auxiliary functions

In [2]:
x1, y1, z1, x2, y2, z2 = symbols("x1, y1, z1, x2, y2, z2", real=True)
xij, yij, zij, dzij, zi = symbols("xij, yij, zij, dzij, zi", real=True)

b, h, eta = symbols("b, h, eta", real=True, positive=True)
theta = symbols("theta", real=True) ##eventually rename theta->psi


mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma = symbols(
    "mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma", integer=True)

C0, C1, C2 = symbols("C0, C1, C2", real=True, positive=True) 
## C0 = 1/(8*pi*eta)
## C1 = theta/(1+theta)
## C2 = (1-theta)/(1+theta)

R1 = Array([x1,y1,z1]) #why not use matrix here?
R2 = Array([x2,y2,z2])

KroneckerDelta = eye(3)

M  = Array([[1,0,0],[0,1,0],[0,0,-1]]) #diag(1,1,-1)

Mf = Array([[C2,0,0], [0, C2,0], [0,0,-1]])
#diag( (1-theta)/(1+theta), (1-theta)/(1+theta), -1) 

R2M = tensorcontraction(tensorproduct(M,R2), (1,2))

rM = R1 - R2M

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)#sqrt(summation(r[alpha]**2,(alpha,0,2)))

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Laplacian(G, r):
    return diff(G, r[0], r[0]) + diff(G, r[1], r[1]) + diff(G, r[2], r[2])


## for some reason I cannot sum over 'symbols' here whenever rM is a Matrix, not an Array. Who knows why, but this seems to work. 

def Curl(mu, beta): ##Curl is always with respect to R1
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(mu,i,j)*diff(G(j,beta),R1[i]) ## Need dependence of G on rM and R2? 
    return result

def tilde_Curl(alpha, sigma): ##tilde_Curl is always with respect to R2
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(sigma,i,j)*diff(G(alpha, j),R2[i])
    return result

def CurlCurl(mu, sigma):
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(mu, i, j)*eps(sigma, k, l)*diff(G(j, l), R1[i], R2[k])
    return result

In [4]:
def Fl(G, l):
    return G + b**2/(4*l+6)*Laplacian(G, R1)

def tilde_Fl(G,l):
    return G + b**2/(4*l+6)*Laplacian(G, R2)

def FlFl(G, l, lprime):
    return G + b**2/(4*l+6)*Laplacian(G, R1) + b**2/(4*lprime+6)*Laplacian(G, R2) + b**4/((4*l+6)*(4*lprime+6))*Laplacian(Laplacian(G, R1), R2)

In [5]:
## Green's function
##first G* part, which is the same for ii and ij, later G0 part for ii

def GO(r, alpha, beta):
    return C0*(KroneckerDelta[alpha,beta]/norm(r) + r[alpha]*r[beta]/norm(r)**3) ##can reuse for particle-particle interactions

## only for Gstar, particle - image interactions
def Stokeslet(alpha, beta):
    result = 0
    for i in range(3):
        result += Mf[beta, i]*GO(rM, alpha, i)
    return result

def Dipole(alpha, beta):
    result = 0
    for i in range(3):
        result += -2*R2[2]*C1*diff(GO(rM, alpha, 2) * M[beta, i], R1[i])
    return result

def Quadrupole(alpha, beta):
    result = 0
    for i in range(3):
        result += R2[2]**2*C1*Laplacian(GO(rM, alpha, i) * M[beta, i], R1)
    return result

def G(alpha, beta): #introduce flag to switch between G0 and Gstar, for whichever we have to compute the matrix elements, also include flag for wall, since theta=infty is hard to handle
    return Stokeslet(alpha, beta) + Dipole(alpha, beta) + Quadrupole(alpha, beta)

In [6]:
## is there not a smarter way to do this? No dependence on rM and R2.. in python want xij, yij etc not x1- x2
## Need: self-interaction and ineraction with image. Interaction with other particle and with its image
## constant, R1=R2 (also constant), rij, and also rij. How to compute all of them with same code? 

## keep dependence on rM and R2: 
        ## for self-interaction choose rM = 0, 0, 2h and R2= 0,0,h
        ## for interaction choose: x1-x2=xij with particle, z1+z2 = ?

## keep dependence for now, might give greater flexibility later - no, delete to make shorter and write extra functions for particle-particle interactions
#for interactions, replace rM by xij, yij, dz = z1 + z2 or so, but there is also zij for particle-particle interactions, need another Greens function GO(rij, alpha, beta)

## could use flag in G, whether to return Go or Gstar

In [7]:
def G1s1s(alpha, beta):
    return simplify((FlFl( G(alpha, beta), 0, 0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2a1s(mu, beta):
    return simplify((b*tilde_Fl(Curl(mu, beta), 0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G1s2a(alpha,sigma):
    return simplify((0.5*b*Fl(tilde_Curl(alpha, sigma), 0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2a2a(mu, sigma):
    return simplify((0.5*b**2*CurlCurl(mu, sigma)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G1s2s(alpha, sigma, eta1):
    return simplify((0.5*b*FlFl((diff(G(alpha,sigma), R2[eta1]) + diff(G(alpha, eta1), R2[sigma])),
                      0, 1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2a2s(mu, sigma, eta1):
    return simplify((0.5*b**2*tilde_Fl( (diff(Curl(mu,sigma), R2[eta1]) + diff(Curl(mu, eta1), R2[sigma])),
                             1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2s1s(mu, nu1, beta):
    return simplify((0.5*b*FlFl( diff( G(mu, beta), R1[nu1]) + diff( G(nu1, beta), R1[mu]),
                      1,0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2s2a(mu, nu1, sigma):
    return simplify((0.25*b**2*Fl( (diff(tilde_Curl(mu,sigma), R1[nu1]) + diff(tilde_Curl(nu1, sigma), R1[mu])),
                        1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2s2s(mu, nu1, sigma, eta1):
    return simplify((0.25*b**2*FlFl( diff( G(mu,sigma), R1[nu1], R2[eta1]) + diff( G(nu1,sigma), R1[mu], R2[eta1])
                          +  diff( G(mu,eta1), R1[nu1], R2[sigma]) + diff( G(nu1,eta1), R1[mu], R2[sigma]),
                          1,1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2s3t(mu, nu1, eta2):
    return simplify((-0.1*b**3*Fl( Laplacian( diff( G(mu, eta2), R1[nu1]) + diff( G(nu1, eta2), R1[mu]), R2),
                        1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2s3a(mu, nu1, sigma, eta2):
    return simplify((1/3*b**3*Fl( diff( diff( tilde_Curl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(mu, beta), R2), (beta,0,2)), R1[nu1]) 
                       + diff( diff( tilde_Curl(nu1, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(nu1, beta), R2), (beta,0,2)), R1[mu]),
                       1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G3a2s(mu, nu2, sigma, nu1):
    return simplify((0.25*b**3*tilde_Fl( diff( diff( Curl(tau, beta), R2[kappa1]) + diff( Curl(tau, kappa1), R2[beta]), R1[gamma2])
                              + diff( diff( Curl(gamma2, beta), R2[kappa1]) + diff( Curl(gamma2, kappa1), R2[beta]), R1[tau]),
                              1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G3a3a(mu, nu2, sigma, eta2):
    return simplify((1/3*b**4*( diff( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(mu, beta), R2), (beta,0,2)), R1[nu2]) 
                       + diff( diff( CurlCurl(nu2, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(nu2, beta), R2), (beta,0,2)), R1[mu]) 
                    )).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G1s3t(alpha, eta2):
    return simplify((-0.2*b**2*Fl( Laplacian( G(alpha, eta2), R2),
                        0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G1s3a(alpha, sigma, eta2):
    return simplify((2/3*b**2*Fl( diff( tilde_Curl(alpha, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( G(alpha,beta), R2), (beta,0,2)),
                       0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2a3a(mu, sigma, eta2):
    return simplify((2/3*b**3*( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( Curl(mu,beta), R2), (beta,0,2)) 
                    )).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G3a1s(mu, nu2, beta):
    return simplify((0.5*b**2*tilde_Fl( diff( Curl(mu,beta), R1[nu2]) + diff( Curl(nu2,beta), R1[mu]),
                             0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G3a2a(mu, nu2, sigma):
    return simplify((0.25*b**3*( diff( CurlCurl(mu,sigma), R1[nu2]) + diff( CurlCurl(nu2,sigma), R1[mu])
                     )).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))


def g13sSym(alpha, sigma, eta1, eta2):
    return 1/3*( diff( G(alpha, sigma), R2[eta1], R2[eta2]) + diff( G(alpha, eta1), R2[eta2], R2[sigma]) + diff( G(alpha, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( G(alpha, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( G(alpha, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( G(alpha, eta1), R2) )

def G1s3s(alpha, sigma, eta1, eta2):
    return simplify((b**2*FlFl( g13sSym(alpha, sigma, eta1, eta2),
                     0, 2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2a3s(mu, sigma, eta1, eta2):
    return simplify((b**3*tilde_Fl( 1/3*( diff( Curl(mu, sigma), R2[eta1], R2[eta2]) + diff( Curl(mu, eta1), R2[eta2], R2[sigma]) + diff( Curl(mu, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( Curl(mu, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( Curl(mu, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( Curl(mu, eta1), R2) ),
                         2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))


def g3s1Sym(alpha, gamma1, gamma2, beta): ##screwed up alpha --> mu etc indexing, so caution with 3s modes below! 
    return 1/3*( diff( G(alpha, beta), R1[gamma1], R1[gamma2]) + diff( G(gamma1, beta), R1[gamma2], R1[alpha]) + diff( G(gamma2, beta), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( G(gamma2, beta), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( G(alpha, beta), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( G(gamma1, beta), R1) )

def G3s1s(mu, nu1, nu2, beta):
    return simplify((b**2*FlFl( g3s1Sym(mu, nu1, nu2, beta),
                     2,0)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G3s2a(alpha, gamma1, gamma2, sigma):
    return simplify((0.5*b**3*Fl( 1/3*( diff( tilde_Curl(alpha, sigma), R1[gamma1], R1[gamma2]) + diff( tilde_Curl(gamma1, sigma), R1[gamma2], R1[alpha]) 
                             + diff( tilde_Curl(gamma2, sigma), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( tilde_Curl(gamma2, sigma), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( tilde_Curl(alpha, sigma), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( tilde_Curl(gamma1, sigma), R1) ),
                       2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def g2s1Sym(mu, nu1, sigma):
    return 0.5*(diff( G(mu, sigma), R1[nu1]) + diff( G(nu1, sigma), R1[mu]))

def G2s3s(mu, nu1, sigma, eta1, eta2):
    return simplify((b**3*FlFl( 1/3*( diff( g2s1Sym(mu, nu1, sigma), R2[eta1], R2[eta2]) + diff( g2s1Sym(mu, nu1, eta1), R2[eta2], R2[sigma]) 
                           + diff( g2s1Sym(mu, nu1, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( g2s1Sym(mu, nu1, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( g2s1Sym(mu, nu1, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( g2s1Sym(mu, nu1, eta1), R2) ),
                     1,2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def g3a1Sym(mu, nu2, sigma):
    return 0.5*( diff( Curl(mu,sigma), R1[nu2]) + diff( Curl(nu2,sigma), R1[mu]) )

def G3a3s(mu, nu2, sigma, eta1, eta2):
    return simplify((b**4*tilde_Fl( 1/3*( diff( g3a1Sym(mu, nu2, sigma), R2[eta1], R2[eta2]) + diff( g3a1Sym(mu, nu2, eta1), R2[eta2], R2[sigma]) 
                           + diff( g3a1Sym(mu, nu2, eta2), R2[sigma], R2[eta1]) ) -  1/15*(
        KroneckerDelta[sigma, eta1]*Laplacian( g3a1Sym(mu, nu2, eta2), R2) + KroneckerDelta[eta1, eta2]*Laplacian( g3a1Sym(mu, nu2, sigma), R2)
        +KroneckerDelta[eta2, sigma]*Laplacian( g3a1Sym(mu, nu2, eta1), R2) ),
                         2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))


def g12sSym(alpha, sigma, eta1):
    return 0.5*( diff(G(alpha,sigma), R2[eta1]) + diff(G(alpha, eta1), R2[sigma]) )

def G3s2s(alpha, gamma1, gamma2, sigma, eta1):
    return simplify((b**3*FlFl( 1/3*( diff( g12sSym(alpha, sigma, eta1), R1[gamma1], R1[gamma2]) + diff( g12sSym(gamma1, sigma, eta1), R1[gamma2], R1[alpha]) 
                           + diff( g12sSym(gamma2, sigma, eta1), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g12sSym(gamma2, sigma, eta1), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g12sSym(alpha, sigma, eta1), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g12sSym(gamma1, sigma, eta1), R1) ),
                     2,1)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))


def g13aSym( alpha, sigma, eta2): 
    return diff( tilde_Curl(alpha, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( G(alpha,beta), R2), (beta,0,2))

def G3s3a(alpha, gamma1, gamma2, sigma, eta2):
    return simplify((2/3*b**4*Fl( 1/3*( diff( g13aSym(alpha, sigma, eta2), R1[gamma1], R1[gamma2]) + diff( g13aSym(gamma1, sigma, eta2), R1[gamma2], R1[alpha]) 
                           + diff( g13aSym(gamma2, sigma, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13aSym(gamma2, sigma, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13aSym(alpha, sigma, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13aSym(gamma1, sigma, eta2), R1) ),
                       2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def g13tSym(alpha, eta2):
    return Laplacian( G(alpha, eta2), R2)

def G3s3t(alpha, gamma1, gamma2, eta2):
    return simplify((-0.2*b**4*Fl( 1/3*( diff( g13tSym(alpha, eta2), R1[gamma1], R1[gamma2]) + diff( g13tSym(gamma1, eta2), R1[gamma2], R1[alpha]) 
                              + diff( g13tSym(gamma2, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13tSym(gamma2, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13tSym(alpha, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13tSym(gamma1, eta2), R1) ),
                        2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G3s3s(alpha, gamma1, gamma2, sigma, eta1, eta2):
    return simplify((b**4*FlFl( 1/3*( diff( g13sSym(alpha, sigma, eta1, eta2), R1[gamma1], R1[gamma2]) 
                           + diff( g13sSym(gamma1, sigma, eta1, eta2), R1[gamma2], R1[alpha]) 
                           + diff( g13sSym(gamma2, sigma, eta1, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
        KroneckerDelta[alpha, gamma1]*Laplacian( g13sSym(gamma2, sigma, eta1, eta2), R1) 
        + KroneckerDelta[gamma1, gamma2]*Laplacian( g13sSym(alpha, sigma, eta1, eta2), R1)
        +KroneckerDelta[gamma2, alpha]*Laplacian( g13sSym(gamma1, sigma, eta1, eta2), R1) ),
                     2,2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G2a3t(mu, eta2):
    return simplify((-0.2*b**3*Laplacian( Curl(mu, eta2),
                               R2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))

def G3a3t(mu, nu2, eta2):
    return simplify((-0.2*b**4*Laplacian( g3a1Sym(mu, nu2, eta2),
                               R2)).subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h, C0:1/(8*pi*eta), C1:theta/(1+theta), C2:(1-theta)/(1+theta) }))


# def k12sSym(alpha, sigma, eta1):
#     return 0.5*( diff( G(alpha, eta1), R2[sigma]) + diff( G(alpha, sigma), R2[eta1]) )

# def K1s2s(alpha, sigma, eta1):
#     return 8*pi*eta*b**2/3*FlFl( k12sSym(alpha, sigma, eta1), 0,1)

# def K1s3t(alpha, eta2):
#     return -6*pi*eta*b**3/25*Fl( Laplacian( G(alpha, eta2), R2), 0)

# def K2a2s(mu, sigma, eta1):
#     return 4*pi*eta*b**3/3*tilde_Fl( diff( Curl(mu, eta1), R2[sigma]) + diff( Curl(mu, sigma), R2[eta1]), 1)


# def k2s1Sym(mu, nu1, sigma):
#     return 0.5*( diff( G(mu, sigma), R1[nu1]) + diff( G(nu1, sigma), R1[mu]))

# def K2s2s(mu, nu1, sigma, eta1):
#     return 4*pi*eta*b**3/3*FlFl( diff( k2s1Sym(mu, nu1, eta1), R2[sigma]) + diff( k2s1Sym(mu, nu1, sigma), R2[eta1]), 1,1)

# def K2s3t(mu, nu1, eta2):
#     return -6*pi*eta*b**4/25*Fl( 0.5*( diff( Laplacian(G(mu, eta2), R2), R1[nu1]) + diff( Laplacian(G(nu1, eta2), R2), R1[mu])), 1)

# def K3a2s(mu, nu2, sigma, eta1):
#     return 2*pi*eta*b**4/3*tilde_Fl( diff( diff( Curl(mu, eta1), R2[sigma]) + diff( Curl(mu, sigma), R2[eta1]), R1[nu2])
#                                     + diff( diff( Curl(nu2, eta1), R2[sigma]) + diff( Curl(nu2, sigma), R2[eta1]), R1[mu]), 1)

# def K3s2s(alpha, gamma1, gamma2, sigma, eta1):
#     return 8*pi*eta*b**4/3*FlFl( 1/3*( diff( k12sSym(alpha, sigma, eta1), R1[gamma1], R1[gamma2]) + diff( k12sSym(gamma1, sigma, eta1), R1[gamma2], R1[alpha]) 
#                            + diff( k12sSym(gamma2, sigma, eta1), R1[alpha], R1[gamma1]) ) -  1/15*(
#         KroneckerDelta[alpha, gamma1]*Laplacian( k12sSym(gamma2, sigma, eta1), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( k12sSym(alpha, sigma, eta1), R1)
#         +KroneckerDelta[gamma2, alpha]*Laplacian( k12sSym(gamma1, sigma, eta1), R1) ), 2,1) 

# def K3s3t(alpha, gamma1, gamma2, eta2): ##up to factor same as G3s3t
#     return -6*pi*eta*b**5/25*Fl( 1/3*( diff( g13tSym(alpha, eta2), R1[gamma1], R1[gamma2]) + diff( g13tSym(gamma1, eta2), R1[gamma2], R1[alpha]) 
#                               + diff( g13tSym(gamma2, eta2), R1[alpha], R1[gamma1]) ) -  1/15*(
#         KroneckerDelta[alpha, gamma1]*Laplacian( g13tSym(gamma2, eta2), R1) + KroneckerDelta[gamma1, gamma2]*Laplacian( g13tSym(alpha, eta2), R1)
#         +KroneckerDelta[gamma2, alpha]*Laplacian( g13tSym(gamma1, eta2), R1) ), 2)

# def K2a3t(mu, eta2):
#     return -6*pi*eta*b**4/25* Laplacian( Curl(mu, eta2), R2)

# def K3a3t(mu, nu2, eta2):
#     return -6*pi*eta*b**5/25*Laplacian( g3a1Sym(mu, nu2, eta2), R2)

In [8]:
## zero elements

def G3t1s(alpha,beta):
    return 0

def G3t2a(alpha,beta):
    return 0

def G3t2s(alpha, beta, kappa1):
    return 0

def G3t3t(alpha,beta):
    return 0

def G3t3a(alpha, beta, kappa1):
    return 0

def G3t3s(alpha, beta, kappa1, kappa2):
    return 0

### Matrix assembly

In [9]:
n_jobs = 30

In [10]:
modes = ['1s', '2a', '2s', '3t', '3a', '3s']
modesV = ['2s', '3t'] ##possible slip modes, for double-layer matrix elements
SLDL  = ['G']
dim   = {'1s':3, '2a':3, '2s':3*3, '3t':3, '3a':3*3, '3s':3*3*3}

In [11]:
%%time
for sldl in SLDL:
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a', '3t']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, kj) for ki in range(3) 
                                                                        for kj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, kj, lj) for ki in range(3)
                                                                            for kj in range(3)
                                                                                for lj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, kj, lj, mj) for ki in range(3)
                                                                                for kj in range(3)
                                                                                    for lj in range(3)
                                                                                        for mj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, li, kj) for ki in range(3) 
                                                                            for li in range(3)
                                                                                for kj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj) for ki in range(3)
                                                                                for li in range(3)
                                                                                    for kj in range(3)
                                                                                        for lj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj, mj) for ki in range(3)
                                                                                    for li in range(3)
                                                                                        for kj in range(3)
                                                                                            for lj in range(3)
                                                                                                for mj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj) for ki in range(3) 
                                                                                for li in range(3)
                                                                                    for mi in range(3)
                                                                                        for kj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj) for ki in range(3)
                                                                                    for li in range(3)
                                                                                        for mi in range(3)
                                                                                            for kj in range(3)
                                                                                                for lj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj, mj) for ki in range(3)
                                                                                        for li in range(3)
                                                                                            for mi in range(3)
                                                                                                for kj in range(3)
                                                                                                    for lj in range(3)
                                                                                                        for mj in range(3))
                    globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                    print(sldl+mode1+mode2+' done')

G1s1s done


KeyboardInterrupt: 

In [12]:
## create double-layer matrices
for mode1 in modes:
    for mode2 in modesV:
        if mode2 == '2s':
            globals()['matK'+mode1+mode2] = 8*pi*eta*b/3 * globals()['matG'+mode1+mode2]
            print('matK'+mode1+mode2+' done')
            
        elif mode2 == '3t':
            globals()['matK'+mode1+mode2] = 6*pi*eta*b/5 * globals()['matG'+mode1+mode2]
            print('matK'+mode1+mode2+' done')

KeyError: 'matG1s2s'

In [ ]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = Matrix(MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = Matrix(MatrixSymbol('torque',3,1))
VH     = Matrix(MatrixSymbol('VH',48,1))
FH     = Matrix(MatrixSymbol('FH',48,1))

Create block matrices

In [ ]:
matG1sH = Matrix(BlockMatrix([matG1s2s, matG1s3t, matG1s3a, matG1s3s]))
matG2aH = Matrix(BlockMatrix([matG2a2s, matG2a3t, matG2a3a, matG2a3s]))

matK1sH = Matrix(BlockMatrix([matK1s2s, matK1s3t, Matrix(ZeroMatrix(3,36))]))
matK2aH = Matrix(BlockMatrix([matK2a2s, matK2a3t, Matrix(ZeroMatrix(3,36))]))

matGH1s = Matrix(BlockMatrix([[matG2s1s],
                              [matG3t1s],
                              [matG3a1s],
                              [matG3s1s]]))
matGH2a = Matrix(BlockMatrix([[matG2s2a],
                              [matG3t2a],
                              [matG3a2a],
                              [matG3s2a]]))

matGHH = Matrix(BlockMatrix([[matG2s2s, matG2s3t, matG2s3a, matG2s3s],
                             [matG3t2s, matG3t3t, matG3t3a, matG3t3s],
                             [matG3a2s, matG3a3t, matG3a3a, matG3a3s],
                             [matG3s2s, matG3s3t, matG3s3a, matG3s3s]]))

matKHH_nonzero = Matrix(BlockMatrix([[matK2s2s, matK2s3t],
                                     [matK3t2s, matK3t3t],
                                     [matK3a2s, matK3a3t],
                                     [matK3s2s, matK3s3t]]))
matKHH = Matrix(BlockMatrix([matKHH_nonzero, Matrix(ZeroMatrix(48,36))]))

Create vectors instead of having to construct the full matrices

In [ ]:
vecGH1sF = matGH1s*force
vecGH2aT = matGH2a*torque

vecKHHVH = matKHH*VH

vecGHHFH = matGHH*FH

vecG1s1sF = matG1s1s*force
vecG2a1sF = matG2a1s*force

vecG1s2aT = matG1s2a*torque
vecG2a2aT = matG2a2a*torque

vecG1sHFH = matG1sH*FH
vecG2aHFH = matG2aH*FH

vecK1sHVH = matK1sH*VH
vecK2aHVH = matK2aH*VH

_____________

### Write symbolic results to file as reusable python functions

In [ ]:
ME_file = "interfaceME.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(h, theta, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file)
    print("def G2a1sF(h, theta, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    
    print("def G1s2aT(h, theta, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("def G2a2aT(h, theta, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    
    print("def GH1sF(gh1sf, h, theta, b,eta, force):", file=text_file)
    print("    gh1sf[:] += {}".format(np.reshape(np.asarray(vecGH1sF),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GH2aT(gh2at, h, theta, b,eta, torque):", file=text_file)
    print("    gh2at[:] += {}".format(np.reshape(np.asarray(vecGH2aT),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KHHVH(khhvh, h, theta, b,eta, VH):", file=text_file)
    print("    khhvh[:] += {}".format(np.reshape(np.asarray(vecKHHVH),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GHHFH(ghhfh, h, theta, b,eta, FH):", file=text_file)
    print("    ghhfh[:] += {}".format(np.reshape(np.asarray(vecGHHFH),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G1sHFH(h, theta, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1sHFH),(3,)).tolist()), file=text_file)
    print("def G2aHFH(h, theta, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2aHFH),(3,)).tolist()), file=text_file)
    
    print("def K1sHVH(h, theta, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK1sHVH),(3,)).tolist()), file=text_file)
    print("def K2aHVH(h, theta, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2aHVH),(3,)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________